In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

pref = '../inaturalist_12K/'
 
train_generator = train_datagen.flow_from_directory(
    pref+'train',
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical')

val_generator = test_datagen.flow_from_directory(
    pref+'val',
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    pref+'test',
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical')

Found 9003 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


# Question 1 : Implementing flexible model 

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from keras.layers import Activation, Dense, Flatten 

class my_model:
    def __init__(self , pretrained_model , input_shape, unfreeze_from):
        unfreeze = int(unfreeze_from)
        self.input_shape = input_shape
        self.model = Sequential()
        pretrained = (pretrained_model(weights = "imagenet", include_top = False, input_shape=input_shape))
        i = 0
        for layers in pretrained.layers[ : (unfreeze)] :
            layers.trainable = False
        for layers in pretrained.layers :
            print(i+1, " " ,layers.name, " ", layers.trainable)
            i = i+1
        self.model.add(pretrained)
        
    def add_dense_layer(self,neurons=10):
        self.model.add(Flatten())
        self.model.add(Dense(neurons,activation='relu'))
        
    def build_model(self,output,optimizer='adam',loss='categorical_crossentropy'):
        self.model.add(Dense(output,activation='softmax'))
        self.model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
        self.model.summary()
        return self.model




In [5]:
model = my_model(ResNet50,(224,224,3),155)
model1 = model.build_model(10)

1   input_1   False
2   conv1_pad   False
3   conv1_conv   False
4   conv1_bn   False
5   conv1_relu   False
6   pool1_pad   False
7   pool1_pool   False
8   conv2_block1_1_conv   False
9   conv2_block1_1_bn   False
10   conv2_block1_1_relu   False
11   conv2_block1_2_conv   False
12   conv2_block1_2_bn   False
13   conv2_block1_2_relu   False
14   conv2_block1_0_conv   False
15   conv2_block1_3_conv   False
16   conv2_block1_0_bn   False
17   conv2_block1_3_bn   False
18   conv2_block1_add   False
19   conv2_block1_out   False
20   conv2_block2_1_conv   False
21   conv2_block2_1_bn   False
22   conv2_block2_1_relu   False
23   conv2_block2_2_conv   False
24   conv2_block2_2_bn   False
25   conv2_block2_2_relu   False
26   conv2_block2_3_conv   False
27   conv2_block2_3_bn   False
28   conv2_block2_add   False
29   conv2_block2_out   False
30   conv2_block3_1_conv   False
31   conv2_block3_1_bn   False
32   conv2_block3_1_relu   False
33   conv2_block3_2_conv   False
34   conv2_block3_

# WandB Sweep 

In [6]:
import wandb
from wandb.keras import WandbCallback

In [7]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model': {
            'values': [ ('ResNet50',174,0) , ('ResNet50',165,1) , ('ResNet50',155,2),('InceptionV3',310,0),('InceptionV3',280,1), ('InceptionV3',249,2) ,('InceptionResNetV2',779,0), ('InceptionResNetV2',742,1),('InceptionResNetV2',710,2) , ('VGG16',18,0), ('VGG16',15,1),('VGG16',11,2)]
        }
    }
}

In [8]:
sweep_id = wandb.sweep(sweep_config,entity = "notarchana" , project = "cs6910-a2")


Create sweep with ID: wyy702q1
Sweep URL: https://wandb.ai/notarchana/cs6910-a2/sweeps/wyy702q1


In [9]:
def get_model(name) :
    if name == "VGG16" :
        return VGG16
    if name == "ResNet50" :
        return ResNet50
    if name == "InceptionV3" :
        return InceptionV3
    if name == "InceptionResNetV2" :
        return InceptionResNetV2

In [10]:
def train():
    config_defaults = {
        'model' : (ResNet50,174)
    }
    wandb.init(config=config_defaults,name="cs6910-a2-partB")
    cfg = wandb.config
    wandb.run.name = cfg.model[0]+ "_" + str(cfg.model[2]) 
    wandb.run.save()
    # Config is a variable that holds and saves hyperparameters and inputs
    print(cfg.model[1])
    model_functor = get_model(cfg.model[0])
    model = my_model( model_functor, (224,224,3) , ((cfg.model)[1]) )
    print("model retrieved")
    model.add_dense_layer(1024)
    model.add_dense_layer(1024)
    model.add_dense_layer(1024)
    model = model.build_model(10)
    
    print("model building done")
    trained = model.fit(train_generator,
                steps_per_epoch=80,
                epochs=10 ,
                batch_size = 64,
                validation_data=val_generator,
                validation_steps=10,
                callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
    )
    
    loss, acc = model.evaluate(test_generator)
    wandb.log({'test_acc ' : acc , 'test_loss ' : loss})
    

    print("model training done")

In [11]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: loj2oxgh with config:
wandb: 	model: ['ResNet50', 174, 0]
wandb: Currently logged in as: notarchana (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


174
1   input_1   False
2   conv1_pad   False
3   conv1_conv   False
4   conv1_bn   False
5   conv1_relu   False
6   pool1_pad   False
7   pool1_pool   False
8   conv2_block1_1_conv   False
9   conv2_block1_1_bn   False
10   conv2_block1_1_relu   False
11   conv2_block1_2_conv   False
12   conv2_block1_2_bn   False
13   conv2_block1_2_relu   False
14   conv2_block1_0_conv   False
15   conv2_block1_3_conv   False
16   conv2_block1_0_bn   False
17   conv2_block1_3_bn   False
18   conv2_block1_add   False
19   conv2_block1_out   False
20   conv2_block2_1_conv   False
21   conv2_block2_1_bn   False
22   conv2_block2_1_relu   False
23   conv2_block2_2_conv   False
24   conv2_block2_2_bn   False
25   conv2_block2_2_relu   False
26   conv2_block2_3_conv   False
27   conv2_block2_3_bn   False
28   conv2_block2_add   False
29   conv2_block2_out   False
30   conv2_block3_1_conv   False
31   conv2_block3_1_bn   False
32   conv2_block3_1_relu   False
33   conv2_block3_2_conv   False
34   conv2_blo

80/80 [==============================] - 423s 5s/step - loss: 2.3035 - accuracy: 0.0957 - val_loss: 2.3025 - val_accuracy: 0.1016
Epoch 10/10
32/32 [==============================] - 136s 4s/step - loss: 2.3026 - accuracy: 0.1000
model training done


epoch,9
loss,2.30293
accuracy,0.09924
val_loss,2.30255
val_accuracy,0.09687
_runtime,4503
_timestamp,1618249396
_step,10
best_val_accuracy,0.14687
best_epoch,0
test_acc,0.1


...

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▂▁▁▁▁
accuracy,▁▅▆▆▇▇▇▇██
val_loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▇█▇█▇▇▇██
_runtime,▁▂▂▃▄▅▅▆▇██
_timestamp,▁▂▂▃▄▅▅▆▇██
_step,▁▂▂▃▄▅▅▆▇▇█
test_acc,▁
test_loss,▁


wandb: Agent Starting Run: 6tpyee38 with config:
wandb: 	model: ['VGG16', 18, 0]


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


18
1   input_1   False
2   block1_conv1   False
3   block1_conv2   False
4   block1_pool   False
5   block2_conv1   False
6   block2_conv2   False
7   block2_pool   False
8   block3_conv1   False
9   block3_conv2   False
10   block3_conv3   False
11   block3_pool   False
12   block4_conv1   False
13   block4_conv2   False
14   block4_conv3   False
15   block4_pool   False
16   block5_conv1   False
17   block5_conv2   False
18   block5_conv3   False
19   block5_pool   True
model retrieved
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________

wandb: Ctrl + C detected. Stopping sweep.
